In [1]:
## Import base classes
import numpy
import random
import sys, os

# For Interactive Plotting
from viz_util import interactive_plotting

# For Environment import 
import sys, os
sys.path.append(os.path.abspath('..'))
from environment.grid import GridEnv, Action

# For Agent import
from multi_ddpg import MADDPG

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 11.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 22.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 2.  0.  0.  0.  0.  0.  0.  1.]]
[(7, 7), (0, 7)]
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 11.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 22.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  0.  0.  1.]]
[(3, 1), (5, 5)]
True


ImportError: cannot import name 'DDPGAgent'

In [9]:
# Create Environment
def create_environment(name):
    if "taxi-grid":
        env = GridEnv()
    
    return state_size, action_size, num_agent = env.observation_space, action_space, num_agent

state_size, action_size, num_agent = create_environment("taxi-grid")

SyntaxError: invalid syntax (<ipython-input-9-51be03551191>, line 6)

In [8]:
# import gym

# env= gym.make('FrozenLake-v0')
# observation = env.reset()
# print(observation)

0


In [7]:
env.step([0,1])

Exception: not an action

In [10]:
actions = Action()

NameError: name 'Action' is not defined

In [ ]:
# Create Environment
agent = MADDPG(state_size=state_size, action_size=action_size, random_seed = 1, num_agent=num_agent)

In [7]:
def DDPG_eval(agent, env, max_t=100):
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    episode_length = 0                                     # measure number of steps allowed before done flag
    for i in range(max_t):
        # actions = np.random.randn(num_agents, action_size) # select an action (for each agent); randomly!
        actions = agent.act(states[0]) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to the environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        episode_length +=1
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Episode length is {}'.format(episode_length))
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [8]:
env.step([Action.East,Action.West])

NameError: name 'env' is not defined

In [ ]:
DDPG_eval(agent, env)